# Test #1 - The influence of geology, faults and fractures

## Initialization

In [1]:
# Loads pyKasso and other useful packages
import copy
import numpy as np
import pyvista as pv
import pykasso as pk
import pykasso.visualization as pkv

pv.global_theme.notebook = False

In [2]:
# Creates a new project directory
project_path = 'geometry_01'
pk.create_project(project_path)

In [3]:
# Constructs the settings
sks_settings = {
    'sks' : {
        'seed' : 1111,
    },
    'grid' : {
        'x0' : 0,
        'y0' : 0,
        'z0' : 50,
        'nx' : 100,
        'ny' : 50,
        'nz' : 50,
        'dx' : 10,
        'dy' : 10,
        'dz' : 10
    },
    'outlets' : {
        'number'     : 1,
        'data'       : [[994, 250]],
        'importance' : [1],
        'mode'       : 'domain_bottom',
    },
    'inlets' : {
        'number'     : 1,
        'data'       : [[-5, 250]],
        'per_outlet' : [1],
        'importance' : [1],
        'mode'       : 'domain_surface',
    },
}

## Easiest case : No domain delimitations + no geological features 

In [4]:
# Computes the karstic networks
for algorithm in ['Isotropic3', 'Riemann3']:
    sks_settings['sks']['algorithm'] = algorithm
    sim = pk.SKS(sks_settings=sks_settings)
    sim.build()
    sim.compute()

In [5]:
# Visualizes the results
visualization = pkv.Visualizer(project_path)
settings = {
    'ghosts'  : [0],
    'outline' : True,
    'inlets'  : True,
    'outlets' : True,
    'cpos'    : 'xz',
}
settings_ = settings.copy()
settings_['outline'] = False
visualization.show([0, 1], ['karst'], settings=settings)

## Influence of the faults

In [6]:
# Constructs the faults
faults = np.full((100,50,50), 0)
faults[:, :, 25] = 1

# Visualizes the faults
# pkv.show_array(faults)

# Updates the settings
new_settings = copy.deepcopy(sks_settings)
new_settings['faults'] = {'data' : faults}

In [7]:
# Computes the karstic networks
for algorithm in ['Isotropic3', 'Riemann3']:
    new_settings['sks']['algorithm'] = algorithm
    sim = pk.SKS(sks_settings=new_settings)
    sim.build()
    sim.compute()

In [8]:
# Visualizes the results
settings = {
    'ghosts'  : [0],
    'outline' : True,
    'inlets'  : True,
    'outlets' : True,
    'cpos'    : 'xz',
}
# visualization.show([2, 3], ['karst', 'faults'], settings=settings)

## Influence of the fracturation

In [9]:
# Declares the fracturation settings
fractures = {
    'seed': 111111,
    'settings': {
        'family_01': {
            'alpha'        : 2,
            'density'      : 0.00005,
            'orientation'  : [90, 120],
            'dip'          : 90,
            'length'       : [300, 500],
        }, 
        'family_02': {
            'alpha'        : 2,
            'density'      : 0.00005,
            'orientation'  : [340,20],
            'dip'          : [0, 90],
            'length'       : [400,600],
        }
    }
}

# Updates the settings
new_settings = copy.deepcopy(sks_settings)
new_settings['fractures'] = fractures

In [10]:
# Computes the karstic networks
for algorithm in ['Isotropic3', 'Riemann3']:
    new_settings['sks']['algorithm'] = algorithm
    sim = pk.SKS(sks_settings=new_settings)
    sim.build()
    sim.compute()

In [11]:
sim.fractures.fractures

,family_id,x,y,z,radius,orientation,dip,normal
0,1,1085.616879,699.204133,758.114736,226.302985,114.969195,90.000000,"(-2.584806449604582e-17, -5.550925164747675e-1..."
1,1,580.889012,332.543086,514.839089,230.738441,101.949688,90.000000,"(-1.267832072465527e-17, -5.990542246122149e-1..."
2,1,356.332719,532.351925,746.243942,152.860164,116.143092,90.000000,"(-2.6979853483997716e-17, -5.4968054019008713e..."
3,1,172.467134,-22.855182,-103.865691,192.315086,103.267445,90.000000,"(-1.4052623305780238e-17, -5.959801368234084e-..."
4,1,1028.131196,531.745430,-204.287175,159.220691,96.492085,90.000000,"(-6.923293101350193e-18, -6.083968663041782e-1..."
...,...,...,...,...,...,...,...,...
67,2,271.546046,689.074839,522.725254,242.420508,9.377170,27.515656,"(0.8750333330706517, -0.14450263252533987, -0...."
68,2,930.107632,-182.253714,670.177368,240.772382,-7.285139,9.448937,"(0.9784691956044534, 0.125086851348958, -0.986..."
69,2,-212.820709,-213.541952,702.756662,203.434217,-7.571692,59.547125,"(0.5024103396017039, 0.06678322183618483, -0.5..."
70,2,326.762613,138.003454,-18.304914,275.989079,1.955171,41.370268,"(0.7500172443828528, -0.02560365071113172, -0...."


In [12]:
# Visualizes the results
settings = {
    'ghosts'  : [0],
    'outline' : True,
    'inlets'  : True,
    'outlets' : True,
    'cpos'    : 'xz',
}
visualization.show([4, 5], ['karst', 'fractures'], settings=settings)

## Influence of the geology

In [13]:
# Constructs the geology
geology = np.full((100,50,50), 1)
geology[:, 35:50, :] = 2

# Visualizes the geology
# pkv.show_array(geology)

# Updates the settings
new_settings = copy.deepcopy(sks_settings)
new_settings['geology'] = {
    'data' : geology,
    'costs' : {1: 0.6, 2: 0.4}
}

In [14]:
# Computes the karstic networks
for algorithm in ['Isotropic3', 'Riemann3']:
    new_settings['sks']['algorithm'] = algorithm
    sim = pk.SKS(sks_settings=new_settings)
    sim.build()
    sim.compute()

In [15]:
# Visualizes the results
settings = {
    'ghosts'  : [0],
    'outline' : True,
    'inlets'  : True,
    'outlets' : True,
    'cpos'    : 'xz',
}
visualization.show([6,7], ['karst', 'cost'], settings=settings)